TODO
- max_length = 
- batch_size = 
- label smoothing
- paramterize tokens with sentencepiece

# Boilerplate

In [17]:
# files
TRAINING_DIRECTORY = '../data/preprocessed_stories'
EXTENSION = '.clean'
N_TRAIN_FILES = 40000
N_TEST_FILES = 500
TOKENS_PER_BATCH = 4096

# tokenization
PAD_TOKEN = 0
INPUT_END_TOKEN = 1
TARGET_END_TOKEN = 2
OOV_TOKEN = 3
MAX_VOCAB_SIZE = 20000

# MODEL_PARAMS
DECODER_INPUT_LEN = 180
DECODER_TARGET_LEN = 20
DECODER_TOTAL_LEN = DECODER_INPUT_LEN + DECODER_TARGET_LEN

In [3]:
# # show tensor2tensor hparams for summarization for reference
# import tensor2tensor.models.transformer
# from tensor2tensor.utils.registry import hparams
# params = hparams('transformer_prepend')()
# for k, v in sorted(vars(params).items(), key=lambda tup: tup[0]):
#     if not k.startswith('_') and not callable(v):
#         print(f'{k}={v}')

# Read in files

In [4]:
import glob

In [5]:
FILES = glob.glob('%s/*%s' % (TRAINING_DIRECTORY, EXTENSION))
print(len(FILES))
FILES

312085


['../data/preprocessed_stories/000064fee589e5607c1534a69f852d37b4936cca.clean',
 '../data/preprocessed_stories/0000800d9058217f6509d7e63ad475e2de0da611.clean',
 '../data/preprocessed_stories/0000bf554ca24b0c72178403b54c0cca62d9faf8.clean',
 '../data/preprocessed_stories/0000dfd9f52a470b9f29957686c2704b68cd0635.clean',
 '../data/preprocessed_stories/000128cbd36642ced67ac90bd7d4d1dd5e8cf554.clean',
 '../data/preprocessed_stories/0001d1afc246a7964130f43ae940af6bc6c57f01.clean',
 '../data/preprocessed_stories/0001d4ce3598e37f20a47fe609736f72e5d73467.clean',
 '../data/preprocessed_stories/0001dc22494415d03319a6833a00cd9c559f1395.clean',
 '../data/preprocessed_stories/0001f1fcec4ca8bc7e278607ba0e31e5cc046e66.clean',
 '../data/preprocessed_stories/0002067d13d3ca304e0bc98d04dde85d4091c55e.clean',
 '../data/preprocessed_stories/0002095e55fcbd3a2f366d9bf92a95433dc305ef.clean',
 '../data/preprocessed_stories/000219931d2c3aae55dc2acdc5f690d0c112ab17.clean',
 '../data/preprocessed_stories/00022dbfa

In [6]:
TRAIN_FILES = FILES[:N_TRAIN_FILES]
TEST_FILES = FILES[N_TRAIN_FILES:N_TRAIN_FILES+N_TEST_FILES]

# Define batch generator

# Initialize Tokenizer

In [7]:
import numpy as np
import sentencepiece as spm

class BytePairEncoder:
    def __init__(self, vocab_size, model_name, *, model_file=None, vocab_file=None,
                 training_file=None, processor=None, **kwargs):
        self.vocab_size = vocab_size
        self.model_name = model_name
        self.training_file = training_file
        self.model_file = f'{self.model_name}.model' if model_file is None else model_file
        self.vocab_file = f'{self.model_name}.vocab' if vocab_file is None else vocab_file
        if processor is None:
            if training_file is None:
                raise ValueError('training_file cannot be None when processor is also None.')
            processor = self._fit(input=training_file, vocab_size=vocab_size,
                                  model_prefix=model_name, model_type='bpe',
                                  **kwargs)
        self.processor = processor
        
    def encode(self, text):
        return np.array(self.processor.EncodeAsIds(text))
    
    def encode_as_pieces(self, text):
        return self.processor.EncodeAsPieces(text)
    
    def decode(self, ids):
        return self.processor.DecodeIds(ids.tolist())
    
    def decode_pieces(self, pieces):
        return self.processor.DecodePieces(pieces)

    @classmethod
    def from_files(cls, model_file, vocab_file):
        model_name = model_file.partition('.')[0]
        processor = cls._load_model(model_file)
        for vocab_size, _ in enumerate(open(vocab_file), start=1): pass
        return cls(vocab_size=vocab_size, model_name=model_name, processor=processor,
                   model_file=model_file, vocab_file=vocab_file)
        
    @staticmethod
    def _load_model(filename):
        processor = spm.SentencePieceProcessor()
        processor.Load(filename)
        return processor
        
    def _fit(self, **kwargs):
        params = ' '.join([f'--{k}={v}' for k, v in kwargs.items()])
        spm.SentencePieceTrainer.Train(params)
        processor = self._load_model(self.model_file)
        return processor

/Users/dgates/venvs/py3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [9]:
# trained in byte-pair-encoding
TOKENIZER = BytePairEncoder.from_files('summarizer.model', 'summarizer.vocab')

In [10]:
TOKENIZER.vocab_size

30000

# Tokenize files

In [13]:
%%time
from tqdm import tqdm_notebook as tqdm
TOKENIZED_FILES = load_files(tqdm(FILES[:100]), TOKENIZER.encode)

HBox(children=(IntProgress(value=0), HTML(value='')))

CPU times: user 451 ms, sys: 16.8 ms, total: 468 ms
Wall time: 494 ms


# Take a look at batches

In [30]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from data import BaseBatchGenerator


class TrainingExample:
    """Simple container to keep track of training data. Useful for debugging."""
    def __init__(self, item, context_text, target_text, context_tokens,
                 target_tokens):
        self.item = item
        self.context_text = context_text
        self.target_text = target_text
        self.context_tokens = context_tokens
        self.target_tokens = target_tokens


def load_files(files, tokenizer):
    """Load and tokenize files."""
    training_examples = []
    for file in files:
        with open(file) as f:
            context_text, target_text = f.read().split('\t')
        context_tokens = tokenizer(context_text)
        target_tokens = tokenizer(target_text)
        example = TrainingExample(file, context_text, target_text,
                                  context_tokens, target_tokens)
        training_examples.append(example)
    return training_examples


class SummaryBatchGenerator(BaseBatchGenerator):
    def __init__(self, max_context_len=None, max_target_len=None, eos_token=-1,
                 pad_token=0, prepend=True):
        self.max_context_len = max_context_len
        self.max_target_len = max_target_len
        self.eos_token = eos_token
        self.pad_token = pad_token
        self.prepend = prepend

    def generate_steps(self, item):
        example = item  # alias
        if self.max_target_len is not None \
                and len(example.target_tokens) > self.max_target_len:
            return []
        if self.max_context_len is not None:
            encoder_tokens = example.context_tokens[:self.max_context_len]
        else:
            encoder_tokens = example.context_tokens
        decoder_tokens = example.target_tokens + [self.eos_token] if self.prepend else []
        decoder_tokens += example.target_tokens + [self.eos_token]
        training_step = encoder_tokens, decoder_tokens
        return [training_step]

    def generate_batches(self, steps, batch_size):
        batches = []
        min_batch_size = 0.95 * batch_size
        max_batch_size = 1.05 * batch_size
        step_sizes = [len(e_toks) + len(d_toks) for e_toks, d_toks in steps]
        current_batch_x1s = []
        current_batch_x2s = []
        current_batch_size = 0
        i = None
        items = enumerate(zip(steps, step_sizes, step_sizes[1:]))
        for i, (step, step_size, next_step_size) in items:
            if step_size > max_batch_size:
                print(f'skipping step with size {step_size}')
                continue
            encoder_tokens, decoder_tokens = step
            current_batch_x1s.append(encoder_tokens)
            current_batch_x2s.append(decoder_tokens)
            current_batch_size += step_size
            if min_batch_size <= current_batch_size + next_step_size <= max_batch_size:
                x1 = pad_sequences(current_batch_x1s, value=self.pad_token)
                x2 = pad_sequences(current_batch_x2s, value=self.pad_token)
                X = [x1, x2[:,:-1]]
                y = x2[:,1:]
                batches.append((X, y))
                current_batch_size = 0
                current_batch_x1s, current_batch_x2s = [], []
            # if there aren't enough steps left to create a full sized batch
            # then break
            if sum(step_sizes[i+1:]) < batch_size:
                break
        return (batches, steps[i+1:]) if i is not None else (batches, steps)


In [37]:
%%time
epoch = SummaryBatchGenerator().generate_epoch(TOKENIZED_FILES, batch_size=TOKENS_PER_BATCH)

CPU times: user 17 µs, sys: 18 µs, total: 35 µs
Wall time: 62.9 µs


In [35]:
next(epoch)

([array([[ 5100,   997, 21425, ...,  1980,  2043,   306]], dtype=int32),
  array([[20080,   526, 59934,  2114, 59854, 59968,  5096,   498, 59886,
           1988, 59996,   458,  8154,  8526,   526, 57426,   488, 59968,
           5096,   498, 59886, 53552, 16648,  5096,   498, 59886, 18914,
             74,  2036, 59900,   994, 59900,   550, 55452,  5096,   498,
            292,  3910,    54,   280,  2206,    58,    14,  2586,   440,
           8584, 23352, 59886,  8138,    74, 15278,   198,  3878, 59886,
             14,   964,  7268, 59886,   434,   666,    58,    14, 15640,
           5082,   184,    14,   786,    74,   434, 58266,   142]],
        dtype=int32)],
 array([[  526, 59934,  2114, 59854, 59968,  5096,   498, 59886,  1988,
         59996,   458,  8154,  8526,   526, 57426,   488, 59968,  5096,
           498, 59886, 53552, 16648,  5096,   498, 59886, 18914,    74,
          2036, 59900,   994, 59900,   550, 55452,  5096,   498,   292,
          3910,    54,   280,  2206, 

In [39]:
x1s, x2s, ys = [], [], []
for X, y in epoch:
    x1, x2 = X
    x1s.append(x1), x2s.append(x2), ys.append(y)
x1 = np.concatenate(x1)
x2 = np.concatenate(x2)
y = np.concatenate(y)

In [40]:
x1.shape, x2.shape, y.shape

((2239,), (71,), (71,))

In [46]:
TOKENIZER.decode(x1)

"Four young cousins enjoyed a night out at a themed restaurant with their grandparents just hours before a deadly blaze ripped through the $9million mansion they were sleeping in. Don Pyle and his wife, Sandra took Alexis (Lexi) Boone, eight, Kaitlyn (Katie) Boone, seven, Charlotte Boone, eight, and Wesley (Wes) Boone, six, to a Medieval Times restaurant in Hannover, Maryland, as a special treat on Sunday night, according to a family spokesman. They had even stopped off at a Target store on the way to pick up themed costumes for the evening. The following morning the 16,000ft waterfront Annapolis property they were sleeping in had been burned to the ground - with all six believed to have been killed. Four unidentified bodies have been pulled out of the wreckage so far while investigators try to determine what caused the fire. Even though remains of all six have not yet been found, all of those inside are believed to have died in the fire. Scroll down for video Unaccounted for: Sisters 

In [45]:
def view_training_examples(x1, x2, y):
    print(' '.join([index_to_word[i] for i in x1]))
    print('\n')
    print(' '.join([index_to_word[i] for i in x2]))
    print('\n')
    print(' '.join([index_to_word[i] for i in y]))

In [47]:
np.where(x2[7] == 1), np.where(y[7] == 1)

((array([], dtype=int64),), (array([], dtype=int64),))

In [48]:
i = np.random.randint(0, len(x1))
examples = x1[i], x2[i], y[i]
view_training_examples(*examples)

IndexError: index 2188 is out of bounds for axis 0 with size 71

# Training

## Define functions for loss/metrics/callbacks

In [ ]:
import tensorflow as tf
import keras.backend as K

# on custom implementation rather than keras see
# https://github.com/tensorflow/tensorflow/issues/17150
def sparse_categorical_crossentropy(y_true, y_pred):
    return tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_true,
                                                          logits=y_pred)

def perplexity(y_true, y_pred):
    cross_entropy = sparse_categorical_crossentropy(y_true, y_pred)
    return K.exp(cross_entropy)

In [ ]:
import numpy as np

# see
# https://github.com/tensorflow/tensor2tensor/blob/master/tensor2tensor/utils/learning_rate.py
class LRScheduler:
    """Stateful learning rate scheduler.
    
    Useful if training is stopped and then resumed so that scheduling
    resumes considering the epoch during which training was interrupted.
    """
    def __init__(self, d_model, warmup_steps, learning_rate):
        self.d_model = d_model
        self.warmup_steps = warmup_steps
        self.learning_rate = learning_rate
        self.epoch = 1
        self.initial_lr = self.lr()
        
    def lr(self, *args):
        scalar = 5000 \
               * self.d_model**-0.5 \
               * min(self.epoch * self.warmup_steps**-1.5, self.epoch**-0.5)
        self.epoch += 1
        return 0.002 * scalar * self.learning_rate

## Define validation set

In [ ]:
%%time
X_test, y_test = SummaryBatchGenerator(
    tokenizer=TOKENIZER,
    num_words=NUM_WORDS,
    decoder_input_len=DECODER_INPUT_LEN,
    decoder_target_len=DECODER_TARGET_LEN,
    input_end_token=INPUT_END_TOKEN,
    target_end_token=TARGET_END_TOKEN,
    pad_token=PAD_TOKEN
).to_arrays(TEST_FILES, batch_size=32)

## Training configs

In [ ]:
from keras.optimizers import adam
from keras.callbacks import LearningRateScheduler

In [ ]:
# # model architecture
N_HEADS = 8
N_LAYERS = 4
D_MODEL = 64*N_HEADS
SENTENCE_LEN = DECODER_TOTAL_LEN
VOCAB_SIZE = NUM_WORDS
DROPOUT = 0.1
OUTPUT_ACTIVATION = 'linear'  # temporary workaround for keras bug - see above

# learning rate
WARMUP_STEPS = 8000
LEARNING_RATE = 0.2
LEARNING_RATE_SCHEDULER = LRScheduler(D_MODEL, WARMUP_STEPS, LEARNING_RATE)

# # optimization
# # https://arxiv.org/pdf/1804.00247.pdf
BETA_1 = 0.9
BETA_2 = 0.98
EPSILON = 1e-9
OPTIMIZER = adam(lr=LEARNING_RATE_SCHEDULER.initial_lr, beta_1=BETA_1, beta_2=BETA_2, epsilon=EPSILON)
METRICS = [sparse_categorical_crossentropy]
LOSS = perplexity

# # batch training
N_EPOCHS = 1000
BATCH_SIZE = 32
CALLBACKS = [LearningRateScheduler(LEARNING_RATE_SCHEDULER.lr)]

## Training

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
learning_rates = [LRScheduler(D_MODEL, WARMUP_STEPS, LEARNING_RATE).lr()
                  for _ in range(WARMUP_STEPS+2000)]
plt.plot(learning_rates)

In [ ]:
from model import Transformer
model = Transformer(
    n_heads=N_HEADS,
    encoder_layers=N_LAYERS,
    decoder_layers=N_LAYERS,
    d_model=D_MODEL,
    vocab_size=VOCAB_SIZE,
    sequence_len=SENTENCE_LEN,
    dropout=DROPOUT,
    output_activation=OUTPUT_ACTIVATION)

In [ ]:
model.summary()

In [ ]:
TARGET_PLACEHOLDER = tf.placeholder(dtype='int32', shape=(None, SENTENCE_LEN))
model.compile(
    loss=LOSS,
    optimizer=OPTIMIZER,
    metrics=METRICS,
    target_tensors=[TARGET_PLACEHOLDER])

In [ ]:
# print loss values for reference
def display_loss_reference():
    format_ = 'perplexity: %20s\tentropy: %20s'
    upper_limit = np.log(VOCAB_SIZE)
    print(format_ % (np.exp(upper_limit), upper_limit))
    for i in reversed(range(int(np.floor(upper_limit))+1)):
        print(format_ % (np.exp(i), i))
display_loss_reference()

In [ ]:
model.fit(
    X, y,
    epochs=N_EPOCHS,
    batch_size=32,
    validation_data=(X_test, y_test),
    callbacks=CALLBACKS)

In [ ]:
# Train on 39552 samples, validate on 480 samples
# Epoch 1/1000
# 39552/39552 [==============================] - 1046s 26ms/step - loss: 20030.8325 - sparse_categorical_crossentropy: 9.9038 - val_loss: 19996.6217 - val_sparse_categorical_crossentropy: 9.9021
# Epoch 2/1000
# 39552/39552 [==============================] - 1038s 26ms/step - loss: 19964.4630 - sparse_categorical_crossentropy: 9.9004 - val_loss: 19909.8440 - val_sparse_categorical_crossentropy: 9.8975
# Epoch 3/1000
# 39552/39552 [==============================] - 1038s 26ms/step - loss: 19875.1671 - sparse_categorical_crossentropy: 9.8956 - val_loss: 19800.6775 - val_sparse_categorical_crossentropy: 9.8915
# Epoch 4/1000
# 39552/39552 [==============================] - 1038s 26ms/step - loss: 19764.9059 - sparse_categorical_crossentropy: 9.8895 - val_loss: 19673.7771 - val_sparse_categorical_crossentropy: 9.8842
# Epoch 5/1000
# 39552/39552 [==============================] - 1038s 26ms/step - loss: 19636.3829 - sparse_categorical_crossentropy: 9.8820 - val_loss: 19527.0184 - val_sparse_categorical_crossentropy: 9.8754
# Epoch 6/1000
# 39552/39552 [==============================] - 1038s 26ms/step - loss: 19489.6364 - sparse_categorical_crossentropy: 9.8733 - val_loss: 19359.5064 - val_sparse_categorical_crossentropy: 9.8653
# Epoch 7/1000
# 39552/39552 [==============================] - 1038s 26ms/step - loss: 19315.6074 - sparse_categorical_crossentropy: 9.8629 - val_loss: 19156.0466 - val_sparse_categorical_crossentropy: 9.8531
# Epoch 8/1000
# 39552/39552 [==============================] - 1038s 26ms/step - loss: 19100.5667 - sparse_categorical_crossentropy: 9.8501 - val_loss: 18898.7443 - val_sparse_categorical_crossentropy: 9.8378
# Epoch 9/1000
# 39552/39552 [==============================] - 1038s 26ms/step - loss: 18821.9885 - sparse_categorical_crossentropy: 9.8336 - val_loss: 18562.3658 - val_sparse_categorical_crossentropy: 9.8178
# Epoch 10/1000
# 39552/39552 [==============================] - 1038s 26ms/step - loss: 18452.5750 - sparse_categorical_crossentropy: 9.8118 - val_loss: 18120.1948 - val_sparse_categorical_crossentropy: 9.7915
# Epoch 11/1000
# 39552/39552 [==============================] - 1039s 26ms/step - loss: 17962.7013 - sparse_categorical_crossentropy: 9.7826 - val_loss: 17541.4365 - val_sparse_categorical_crossentropy: 9.7564
# Epoch 12/1000
# 39552/39552 [==============================] - 1038s 26ms/step - loss: 17321.3350 - sparse_categorical_crossentropy: 9.7435 - val_loss: 16791.8242 - val_sparse_categorical_crossentropy: 9.7094
# Epoch 13/1000
# 39552/39552 [==============================] - 1038s 26ms/step - loss: 16493.9197 - sparse_categorical_crossentropy: 9.6909 - val_loss: 15833.8628 - val_sparse_categorical_crossentropy: 9.6463
# Epoch 14/1000
# 39552/39552 [==============================] - 1038s 26ms/step - loss: 15456.5416 - sparse_categorical_crossentropy: 9.6210 - val_loss: 14659.6625 - val_sparse_categorical_crossentropy: 9.5631
# Epoch 15/1000
# 39552/39552 [==============================] - 1038s 26ms/step - loss: 14218.1455 - sparse_categorical_crossentropy: 9.5304 - val_loss: 13308.3386 - val_sparse_categorical_crossentropy: 9.4573
# Epoch 16/1000
# 39552/39552 [==============================] - 1038s 26ms/step - loss: 12833.7705 - sparse_categorical_crossentropy: 9.4174 - val_loss: 11862.0946 - val_sparse_categorical_crossentropy: 9.3286
# Epoch 17/1000
# 39552/39552 [==============================] - 1039s 26ms/step - loss: 11394.6502 - sparse_categorical_crossentropy: 9.2828 - val_loss: 10424.0598 - val_sparse_categorical_crossentropy: 9.1791
# Epoch 18/1000
# 39552/39552 [==============================] - 1038s 26ms/step - loss: 9995.9602 - sparse_categorical_crossentropy: 9.1289 - val_loss: 9081.8747 - val_sparse_categorical_crossentropy: 9.0117
# Epoch 19/1000
# 39552/39552 [==============================] - 1039s 26ms/step - loss: 8710.7632 - sparse_categorical_crossentropy: 8.9581 - val_loss: 7892.9458 - val_sparse_categorical_crossentropy: 8.8290
# Epoch 20/1000
# 39552/39552 [==============================] - 1039s 26ms/step - loss: 7585.5990 - sparse_categorical_crossentropy: 8.7730 - val_loss: 6887.4789 - val_sparse_categorical_crossentropy: 8.6337
# Epoch 21/1000
# 39552/39552 [==============================] - 1039s 26ms/step - loss: 6644.8232 - sparse_categorical_crossentropy: 8.5763 - val_loss: 6077.2281 - val_sparse_categorical_crossentropy: 8.4287
# Epoch 22/1000
# 39552/39552 [==============================] - 1040s 26ms/step - loss: 5896.3155 - sparse_categorical_crossentropy: 8.3711 - val_loss: 5461.0650 - val_sparse_categorical_crossentropy: 8.2179
# Epoch 23/1000
# 39552/39552 [==============================] - 1039s 26ms/step - loss: 5336.9727 - sparse_categorical_crossentropy: 8.1625 - val_loss: 5030.3200 - val_sparse_categorical_crossentropy: 8.0075
# Epoch 24/1000
# 39552/39552 [==============================] - 1040s 26ms/step - loss: 4955.3694 - sparse_categorical_crossentropy: 7.9588 - val_loss: 4767.2497 - val_sparse_categorical_crossentropy: 7.8075
# Epoch 25/1000
# 39552/39552 [==============================] - 1040s 26ms/step - loss: 4726.6126 - sparse_categorical_crossentropy: 7.7714 - val_loss: 4635.1207 - val_sparse_categorical_crossentropy: 7.6312
# Epoch 26/1000
# 39552/39552 [==============================] - 1039s 26ms/step - loss: 4610.0907 - sparse_categorical_crossentropy: 7.6136 - val_loss: 4586.8533 - val_sparse_categorical_crossentropy: 7.4910
# Epoch 27/1000
# 39552/39552 [==============================] - 1039s 26ms/step - loss: 4560.1914 - sparse_categorical_crossentropy: 7.4963 - val_loss: 4576.6557 - val_sparse_categorical_crossentropy: 7.3952
# Epoch 28/1000
# 39552/39552 [==============================] - 1040s 26ms/step - loss: 4540.0475 - sparse_categorical_crossentropy: 7.4238 - val_loss: 4578.0855 - val_sparse_categorical_crossentropy: 7.3448
# Epoch 29/1000
# 39552/39552 [==============================] - 1041s 26ms/step - loss: 4529.7697 - sparse_categorical_crossentropy: 7.3891 - val_loss: 4578.7563 - val_sparse_categorical_crossentropy: 7.3238
# Epoch 30/1000
# 39552/39552 [==============================] - 1041s 26ms/step - loss: 4521.7314 - sparse_categorical_crossentropy: 7.3736 - val_loss: 4578.2458 - val_sparse_categorical_crossentropy: 7.3153
# Epoch 31/1000
# 39552/39552 [==============================] - 1042s 26ms/step - loss: 4514.6274 - sparse_categorical_crossentropy: 7.3657 - val_loss: 4577.9977 - val_sparse_categorical_crossentropy: 7.3126
# Epoch 32/1000
# 39552/39552 [==============================] - 1042s 26ms/step - loss: 4507.3862 - sparse_categorical_crossentropy: 7.3605 - val_loss: 4576.8984 - val_sparse_categorical_crossentropy: 7.3104
# Epoch 33/1000
#  4672/39552 [==>...........................] - ETA: 15:14 - loss: 4500.4551 - sparse_categorical_crossentropy: 7.3590
